In [ ]:
dirname = 'logs/thumbnails1'

In [ ]:
from duckietown_utils import locate_files
locate_files(dirname, 'image-*.jpg')

In [ ]:
from duckietown_utils import image_cv_from_jpg_fn
def load_images_rgb(dirname):
    files = locate_files(dirname, 'image-*jpg')
    files = sorted(files)
    images = [image_cv_from_jpg_fn(f) for f in files]
    images = [rgb_from_bgr(_) for _ in images]
    return images

In [ ]:
images = load_images_rgb(dirname)

In [ ]:
print('Loaded %d images; resolution is %s' % (len(images), images[0].shape))

In [ ]:
from matplotlib import pyplot
import numpy as np

A = images[0]
B = images[1]

diff = 128 + (A.astype('float') - B.astype('float'))/2
diff = diff.astype('uint8')

x = np.hstack((A, B, diff))

pyplot.imshow(x);
pyplot.show()

In [ ]:
def get_similarity_matrix(images, phi, distance):
    n = len(images)    
    descriptors = [phi(image) for image in images]
    A = np.zeros((n,n))
    interval = n / 10
    for i in range(n):
        if i % interval == 0:
            print('%d/%d' % (i, n))
        for j in range(n):
            if i <= j:
                d = distance(descriptors[i], descriptors[j])
                A[i,j] = d
                A[j,i] = d
    return A

In [ ]:

from duckietown_utils import d8_image_zoom_linear

def phi_resize(x):
    x = d8_image_zoom_linear(x, 1/16.0)
    return x.astype('float32')

def distance_L1(a,b):
    return np.mean(np.fabs(a-b))


In [ ]:
A = get_similarity_matrix(images, phi_resize, distance_L1)

In [ ]:
pyplot.figure()
pyplot.imshow(A)
pyplot.show()

In [ ]:
pyplot.figure()
pyplot.plot(A[100,:])
pyplot.show()